# PREPROCESS
- Special Treaments
    - delete constant feature: Utility
    - handling categorical features with order
    - binning MSSubClass, Neighbor
- Missing Values
- Outliers
- Handling Categorical Values (Get Dummies)
- Scale
- Save

In [1]:
# -*-coding:utf-8 -*-
import pandas as pd
from os import path
from sklearn.linear_model import LinearRegression, Ridge, Lasso
#import warnings
import numpy as np
#from numpy import sqrt, mean, log
from random import seed, randrange
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [2]:
#LOAD DATA
curdir = path.curdir
train_file = path.join(curdir, "input/train.csv.gz")
test_file = path.join(curdir, "input/test.csv.gz")

train = pd.read_csv(train_file, compression="gzip", index_col="Id", 
                    dtype={"MSSubClass": object, "MoSold":object})
test = pd.read_csv(test_file, compression="gzip", index_col="Id", 
                   dtype={"MSSubClass": object, "MoSold":object})

tr_y = train.loc[:, "SalePrice"].copy()
tr_y = np.log(tr_y) #log(SalePrice)
tr_X = train.iloc[:, :-1].copy()
del train

#ad flag
tr_X.loc[:, "is_test"] = 0
test.loc[:, "is_test"] = 1 
column_order = list(tr_X.columns)

#concat (get_dummies가 train, test모두 동일하게 처리하도록)
data = pd.concat([tr_X, test]).copy()

del tr_X, test

In [3]:
data = data[column_order]
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,is_test
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,0


# special treaments
- delete constant feature: Utility
- handling categorical features with order
- binning MSSubClass, Neighbor

In [4]:
#delete constant feature
data.drop("Utilities", axis=1, inplace=True)
column_order.remove("Utilities")

#process quality type features
categorical_feature_with_order = ["OverallQual",
                                "OverallCond",
                                "ExterQual",
                                "ExterCond",
                                "BsmtQual",
                                "BsmtCond",
                                "BsmtExposure",
                                "BsmtFinType1",
                                "BsmtFinType2",
                                "HeatingQC",
                                "KitchenQual",
                                "FireplaceQu",
                                "GarageQual",
                                "GarageCond",
                                "PoolQC"]

qual_default = {"NA":0, "Po":1, "Fa":2, "TA":3, "Gd":4, "Ex":5}
qual_exp = {"NA":0, "No":1, "Mn":2, "Av":3, "Gd":4}
qual_fin = {"NA":0, "Unf":1, "LwQ":2, "Rec":3, "BLQ":4, "ALQ":5, "GLQ":6}

data.loc[:, tuple(categorical_feature_with_order[2:])] = data.loc[:, tuple(categorical_feature_with_order[2:])].fillna("NA")
for c in categorical_feature_with_order[2:]:
    if c == "BsmtExposure":
        qual_type = qual_exp
    elif c in ["BsmtFinType1", "BsmtFinType2"]:
        qual_type = qual_fin
    else:
        qual_type =qual_default
    data.replace({c: qual_type}, inplace=True)
    
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,is_test
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,Inside,Gtl,...,0,0,NaN,NaN,0,2,2008,WD,Normal,0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,FR2,Gtl,...,0,0,NaN,NaN,0,5,2007,WD,Normal,0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,Inside,Gtl,...,0,0,NaN,NaN,0,9,2008,WD,Normal,0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,Corner,Gtl,...,0,0,NaN,NaN,0,2,2006,WD,Abnorml,0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,FR2,Gtl,...,0,0,NaN,NaN,0,12,2008,WD,Normal,0


In [5]:
def process_MSSubClass(data):
    data.loc[:, "MSSubClass_age"] = data.MSSubClass.copy()
    data.loc[:, "MSSubClass_finish"] = data.MSSubClass.copy()

    replacement_age = {"MSSubClass_age":{"20": "NEWER", 
                                    "30": "OLDER",
                                    "40": "ALL",
                                    "45": "ALL",
                                    "50": "ALL",
                                    "60": "NEWER",
                                    "70": "OLDER", 
                                    "75": "ALL",
                                    "80": "ALL", 
                                    "85": "ALL",
                                    "90": "ALL", 
                                    "120": "NEWER",
                                    "150": "ALL", 
                                    "160":"NEWER", 
                                    "180":"ALL", 
                                    "190": "ALL"}}

    replacement_finish = {"MSSubClass_finish":{"20": "NA", 
                                    "30": "NA",
                                    "40": "FIN",
                                    "45": "UNFIN",
                                    "50": "FIN",
                                    "60": "NA",
                                    "70": "NA", 
                                    "75": "NA",
                                    "80": "NA",
                                    "85": "NA",
                                    "90": "NA", 
                                    "120": "PUD",
                                    "150": "PUD", 
                                    "160":"PUD", 
                                    "180":"PUD", 
                                    "190": "FAM"}}

    data.replace(replacement_age, inplace=True)
    data.replace(replacement_finish, inplace=True)
    data.drop("MSSubClass", axis=1, inplace=True)
    return data
data = process_MSSubClass(data)
data.head()

,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,is_test,MSSubClass_age,MSSubClass_finish
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,65.0,8450,Pave,NaN,Reg,Lvl,Inside,Gtl,CollgCr,...,NaN,NaN,0,2,2008,WD,Normal,0,NEWER,NA
2,RL,80.0,9600,Pave,NaN,Reg,Lvl,FR2,Gtl,Veenker,...,NaN,NaN,0,5,2007,WD,Normal,0,NEWER,NA
3,RL,68.0,11250,Pave,NaN,IR1,Lvl,Inside,Gtl,CollgCr,...,NaN,NaN,0,9,2008,WD,Normal,0,NEWER,NA
4,RL,60.0,9550,Pave,NaN,IR1,Lvl,Corner,Gtl,Crawfor,...,NaN,NaN,0,2,2006,WD,Abnorml,0,OLDER,NA
5,RL,84.0,14260,Pave,NaN,IR1,Lvl,FR2,Gtl,NoRidge,...,NaN,NaN,0,12,2008,WD,Normal,0,NEWER,NA


In [6]:
def _process_infreq_neighbor(value):
    if value in ['IDOTRR', 'OldTown', 'SWISU', 'Crawfor', 'Edwards', 'NAmes',
       'SWISU', 'SawyerW', 'NPkVill', 'Blmngtn', 'Blueste', 'CollgCr',
       'Crawfor', 'Edwards', 'Mitchel', 'NridgHt', 'Sawyer']:
        return "Etc"
    else:
        return value
def process_neighborhood(data):
    data.loc[:, "Neighborhood"] = data.loc[:, "Neighborhood"].apply(lambda x: 
                                                                   _process_infreq_neighbor(x))
    return data
data = process_neighborhood(data)

In [7]:
data.groupby(["MSZoning", "Neighborhood"])["LotFrontage"].count()

MSZoning  Neighborhood
C (all)   Etc               23
FV        Somerst          119
RH        Etc               23
RL        BrkSide           34
          ClearCr           20
          Etc             1373
          Gilbert          111
          NWAmes            85
          NoRidge           54
          Somerst           43
          StoneBr           46
          Timber            57
          Veenker           16
RM        BrDale            30
          BrkSide           61
          Etc              302
          MeadowV           33
Name: LotFrontage, dtype: int64

In [8]:
# generate meta data
def get_meta_data(data):
    meta_data = pd.DataFrame({"type": data.dtypes})
    meta_data.loc[:, "na_count"] = data.isnull().sum()
    for row in meta_data.iterrows():
        if row[1].type in [int, float]:
            meta_data.loc[row[0], "tr_max"] = data.loc[data.is_test == 0, row[0]].max()
            meta_data.loc[row[0], "tr_min"] = data.loc[data.is_test == 0, row[0]].min()
            meta_data.loc[row[0], "tr_mean"] = data.loc[data.is_test == 0, row[0]].mean()
        elif row[1].type == object:
            meta_data.loc[row[0], "tr_top"] = data.loc[data.is_test == 0, row[0]].describe().top
    return meta_data

meta_data = get_meta_data(data)
meta_data

,type,na_count,tr_top,tr_max,tr_min,tr_mean
MSZoning,object,4,RL,NaN,NaN,NaN
LotFrontage,float64,486,NaN,313.0,21.0,70.049958
LotArea,int64,0,NaN,215245.0,1300.0,10516.828082
Street,object,0,Pave,NaN,NaN,NaN
Alley,object,2721,Grvl,NaN,NaN,NaN
LotShape,object,0,Reg,NaN,NaN,NaN
LandContour,object,0,Lvl,NaN,NaN,NaN
LotConfig,object,0,Inside,NaN,NaN,NaN
LandSlope,object,0,Gtl,NaN,NaN,NaN
Neighborhood,object,0,Etc,NaN,NaN,NaN


# Handling Missing Values

In [9]:
def letsfillna(data, meta_data):
    #특정한 값으로 대체하는 경우
    data.loc[:, "LotFrontage"].fillna(0, inplace=True)
    data.loc[:, "GarageYrBlt"].fillna(meta_data.loc["GarageYrBlt"].tr_mean, inplace=True)
    #data.loc[data.loc[:, "GarageYrBlt"] > 2016, "GarageYrBlt"] = 2016
    
    #"NA"로 대체하는 경우
    
    
    #MEAN값 및 최빈값으로 대체하는 경우
    #50개 이하 항목은 categorical항목은 최빈값, numeric 항목은 모두 mean값으로 처리. 따로 항목으로 만들기엔 너무 수가 적음.
    less_than_50 = ['MSZoning', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
                'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual',
                'Functional', 'GarageCars', 'GarageArea', 'SaleType']
    for col in less_than_50:
        if meta_data.loc[col].type == object:
            data.loc[:, col].fillna(meta_data.loc[col].tr_top, inplace=True)
        elif meta_data.loc[col].type in [float, int]:
            data.loc[:, col].fillna(meta_data.loc[col].tr_mean, inplace=True)
    return 
letsfillna(data, meta_data)

# Handling Outliers

In [10]:
#HANDLING OUTLIERS
data.loc[data.LotArea > data.LotArea.mean() + 3 * data.LotArea.std(),
        "LotArea"] = data.LotArea.mean() + 3 * data.LotArea.std()

data.loc[:, "hasWoodDeck"] = data.loc[:, "WoodDeckSF"].apply(
    lambda x: 1 if x > 0 else 0)
data.loc[:, "hasOpenPorch"] = data.loc[:, "OpenPorchSF"].apply(
    lambda x: 1 if x > 0 else 0)
data.loc[:, "hasEnclPorch"] = data.loc[:, "EnclosedPorch"].apply(
    lambda x: 1 if x > 0 else 0)

# Handling Categorical Values (Get Dummies!)

In [11]:
#HANDLING CATEGORICAL VARIABLES
dum_data = pd.get_dummies(data, dummy_na=True, drop_first=True)
del data

In [12]:
dum_data.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleCondition_Partial,SaleCondition_nan,MSSubClass_age_NEWER,MSSubClass_age_OLDER,MSSubClass_age_nan,MSSubClass_finish_FIN,MSSubClass_finish_NA,MSSubClass_finish_PUD,MSSubClass_finish_UNFIN,MSSubClass_finish_nan
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450.0,7,5,2003,2003,196.0,4,3,4,...,0,0,1,0,0,0,1,0,0,0
2,80.0,9600.0,6,8,1976,1976,0.0,3,3,4,...,0,0,1,0,0,0,1,0,0,0
3,68.0,11250.0,7,5,2001,2002,162.0,4,3,4,...,0,0,1,0,0,0,1,0,0,0
4,60.0,9550.0,7,5,1915,1970,0.0,3,3,3,...,0,0,0,1,0,0,1,0,0,0
5,84.0,14260.0,8,5,2000,2000,350.0,4,3,4,...,0,0,1,0,0,0,1,0,0,0


In [13]:
#split into train, test data
dum_train = dum_data.loc[dum_data.is_test == 0].copy()
dum_test = dum_data.loc[dum_data.is_test == 1].copy()
del dum_data
print(dum_train.shape, dum_test.shape)

(1460, 248) (1459, 248)


# Scale!

In [14]:
#SCALE (실제 상황에서 trian, test를 함께 mean, max, min을 구할 수 없는 상황이므로 따로 구한다.)
def scale_features(data, meta_data):
    for row in meta_data.iterrows():
        feature = row[0]
        if row[1].type in [int, float] and feature not in ["Id", "SalePrice", "is_test"]:
            max_val = row[1].tr_min
            min_val = row[1].tr_max
            mean_val = row[1].tr_mean
            print("{} ({})        max: {} min: {} mean: {}".format(feature, row[1].type, max_val, min_val, mean_val))
            data.loc[:, feature] = data.loc[:, feature].apply(lambda val: (val - mean_val)/(max_val - min_val))                
    return data

dum_train = scale_features(dum_train, meta_data)
dum_test = scale_features(dum_test, meta_data)

tr_X = dum_train.drop("is_test", axis=1)
te_X = dum_test.drop("is_test", axis=1)
del dum_train, dum_test

LotFrontage (float64)        max: 21.0 min: 313.0 mean: 70.04995836802665
LotArea (int64)        max: 1300.0 min: 215245.0 mean: 10516.828082191782
OverallQual (int64)        max: 1.0 min: 10.0 mean: 6.0993150684931505
OverallCond (int64)        max: 1.0 min: 9.0 mean: 5.575342465753424
YearBuilt (int64)        max: 1872.0 min: 2010.0 mean: 1971.267808219178
YearRemodAdd (int64)        max: 1950.0 min: 2010.0 mean: 1984.8657534246574
MasVnrArea (float64)        max: 0.0 min: 1600.0 mean: 103.68526170798899
ExterQual (int64)        max: 2.0 min: 5.0 mean: 3.3958904109589043
ExterCond (int64)        max: 1.0 min: 5.0 mean: 3.0835616438356164
BsmtQual (int64)        max: 0.0 min: 5.0 mean: 3.489041095890411
BsmtCond (int64)        max: 0.0 min: 4.0 mean: 2.934931506849315
BsmtExposure (int64)        max: 0.0 min: 4.0 mean: 1.63013698630137
BsmtFinType1 (int64)        max: 0.0 min: 6.0 mean: 3.5458904109589042
BsmtFinSF1 (float64)        max: 0.0 min: 5644.0 mean: 443.6397260273973
BsmtFin

# Save

In [15]:
print(tr_X.shape, te_X.shape)

(1460, 247) (1459, 247)


In [16]:
#save
set_ver = 1
tr_X.to_csv("cache/tr_X_{}.csv.gz".format(set_ver), compression="gzip", index=True)
pd.DataFrame(tr_y).to_csv("cache/tr_y_{}.csv.gz".format(set_ver), compression="gzip", index=True)
te_X.to_csv("cache/te_X_{}.csv.gz".format(set_ver), compression="gzip", index=True)